In [224]:

from asyncore import read
import csv
from itertools import count
import requests
import pandas as pd
from statistics import mean
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import time
import numpy
import keras
import os

def get_data(name):
        # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    CSV_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+name+'&interval=15min&slice=year1month1&apikey=FVENHA1GERNRSPQ4'

    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)
    return pd.DataFrame(data=my_list[1:] ,columns=my_list[0])


def download_and_save_data_to_file(name):
  get_data(name).to_csv(name+".csv",index=False,sep=";")

def mymean(input):
  return mean([float(input['open']),float(input['high']),float(input['low']),float(input['close'])])

def prepare_data(data,number_ofcolumns):
  newdata=pd.DataFrame()
  data["volume"]=data.apply(lambda inputrow:mymean(inputrow),axis=1)
  #print(data)
  data=data[["time","volume"]]
  
  for i in range(int(len(data)/number_ofcolumns)):
    part=data.iloc[i*number_ofcolumns:(i+1)*number_ofcolumns]
    part.index=[0]*len(part)
    part=pd.pivot(part,columns="time",values="volume")
    #part.columns=["1","2","3","4","5","6","7","8","9","10"]
    part.columns=[str(i) for i in range(number_ofcolumns)]
    newdata=newdata.append(part)
    newdata.index=range(len(newdata))
  return newdata.drop_duplicates()

def read_and_prepare_multiple_companies_data(companies,number_of_columns):
  final_data=pd.DataFrame()
  for company in companies:
    
    final_data=final_data.append(prepare_data(get_data(company),number_of_columns))
    time.sleep(3)

  return final_data

def download_and_save_multiple_companies_to_files(companies):
  for company in companies:
    download_and_save_data_to_file(company)

def read_and_orepare_multiple_companies_from_files(number_of_columns):
  data=[]
  counter=0
  for root,dir,files in os.walk(os.getcwd()):    
    for file in files:
      if(file.endswith(".csv")):
        temp=pd.read_csv(file,sep=";")
        data.append(prepare_data(temp,number_of_columns))

  return pd.concat(data, axis=0, ignore_index=True)



    
  


In [216]:
data=pd.read_csv("SAP.csv",sep=";")


In [135]:
companies=['SAP','IBM','GOOG','META','TSLA','AMZN']

In [136]:
download_and_save_multiple_companies_to_files(companies)

In [225]:
sample=read_and_orepare_multiple_companies_from_files(10)

In [226]:
print(sample)

              0           1          2           3           4          5  \
0    121.960000  122.077500  122.32375  122.466250  122.625000  122.74000   
1    123.002500  122.793600  122.83625  122.930075  122.744875  122.54275   
2    124.120000  124.189175  124.31125  124.090000  123.757500  123.78750   
3    125.500000  125.350000  125.00000  124.610000  124.767500  124.39360   
4    125.907500  126.290000  126.19250  125.997500  125.736675  125.38500   
..          ...         ...        ...         ...         ...        ...   
486   84.570000   84.747500   84.60000   84.450000   84.383750   84.24625   
487   85.685000   85.610000   85.22750   84.950000   84.705000   84.66750   
488   84.816250   84.858300   84.83410   84.768750   84.674000   85.07000   
489   84.654550   84.645000   84.82250   84.824525   84.733975   84.85675   
490   84.600025   84.555000   84.76500   84.553975   84.331950   84.30750   

              6           7          8           9  
0    122.850000  123.0

In [227]:
sample.tail(3)

,0,1,2,3,4,5,6,7,8,9
488,84.816250,84.8583,84.8341,84.768750,84.674000,85.07000,86.07000,86.220000,86.2075,85.85000
489,84.654550,84.6450,84.8225,84.824525,84.733975,84.85675,84.93625,84.930775,84.8150,84.86500
490,84.600025,84.5550,84.7650,84.553975,84.331950,84.30750,84.38875,84.417500,84.4350,84.62125


In [154]:
len(sample.columns)

0

In [228]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)   

In [229]:
regressor.fit(sample[[str(i )for i in range(len(sample.columns)-1)]]  ,sample[str(len(sample.columns)-1)])  

RandomForestRegressor(random_state=0)

In [237]:
test=numpy.array(sample.iloc[341].tolist())

In [238]:
regressor.predict([test[0:9]])

array([103.922175])

In [240]:
sample.iloc[341]

0    103.667500
1    103.685000
2    103.767500
3    103.717500
4    103.727525
5    103.792500
6    103.880000
7    103.912500
8    103.907500
9    103.980000
Name: 341, dtype: float64